*Copyright (c) Microsoft Corporation. All rights reserved.*  
*Licensed under the MIT License.*
# Named Entity Recognition Using BERT on Chinese
## Summary
This notebook demonstrates how to fine tune [pretrained BERT model](https://github.com/huggingface/pytorch-pretrained-BERT) for named entity recognition (NER) task on Chinese text. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring and model evaluation.

[BERT (Bidirectional Transformers for Language Understanding)](https://arxiv.org/pdf/1810.04805.pdf) is a powerful pre-trained lanaguage model that can be used for multiple NLP tasks, including text classification, question answering, named entity recognition, etc. It's able to achieve state of the art performance with only a few epochs of fine tuning on task specific datasets.  
The figure below illustrates how BERT can be fine tuned for NER tasks. The input data is a list of tokens representing a sentence. In the training data, each token has an entity label. After fine tuning, the model predicts an entity label for each token in a given testing sentence. 

<img src="https://nlpbp.blob.core.windows.net/images/bert_architecture.png">

Named Entity Recognition on non-English text is not very differnt from that on English text. The only difference is the model used, which is configured by the `LANGUAGE` variable below. For non-English languages including Chinese, the *bert-base-multilingual-cased* model can be used by setting `LANGUAGE = Language.MULTILINGUAL`. For Chinese, the *bert-base-chinese* model can also be used by setting `LANGUAGE = Language.CHINESE`. On Chinese text, the performance of *bert-base-chinese* is usually better than *bert-base-multilingual-cased* because the *bert-base-chinese* model is pretrained on Chinese data only. 

## Required packages
* pytorch
* pytorch-pretrained-bert
* pandas
* seqeval

In [1]:
import sys
import os
import random
from seqeval.metrics import classification_report

import torch

nlp_path = os.path.abspath('../../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.bert.token_classification import BERTTokenClassifier, postprocess_token_labels, create_label_map
from utils_nlp.bert.common import Language, Tokenizer
from utils_nlp.dataset.msra_ner import load_pandas_df, get_unique_labels

## Configurations

In [2]:
# path configurations
CACHE_DIR = "./temp"

# set random seeds
RANDOM_SEED = 100
torch.manual_seed(RANDOM_SEED)

# model configurations
LANGUAGE = Language.CHINESE
DO_LOWER_CASE = True
MAX_SEQ_LENGTH = 200

# training configurations
BATCH_SIZE = 16
NUM_TRAIN_EPOCHS = 1

# optimizer configuration
LEARNING_RATE = 3e-5

TEXT_COL = "sentence"
LABEL_COL = "labels"

## Preprocess Data

### Get training and testing data
The dataset used in this notebook is the MSRA NER dataset. The dataset consists of 45000 training sentences and 3940 testing sentences. 

The helper function `load_pandas_df` downloads the data files if they don't exist in `local_cache_path`. It returns the training or testing data frame based on `file_split`

The helper function `get_unique_labels` returns the unique entity labels in the dataset. There are 7 unique labels in the   dataset: 
* 'O': non-entity 
* 'B-LOC': beginning of location entity
* 'I-LOC': within location entity
* 'B-PER': beginning of person entity
* 'I-PER': within person entity
* 'B-ORG': beginning of organization entity
* 'I-ORG': within organization entity

The maximum number of words in a sentence is 756. We set MAX_SEQ_LENGTH to 200 above to reduce the GPU memory needed to run this notebook. Less than 1% of testing data are longer than 200, so this should have negligible impact on the model performance evaluation.

In [3]:
train_df = load_pandas_df(local_cache_path=CACHE_DIR, file_split="train")
test_df = load_pandas_df(local_cache_path=CACHE_DIR, file_split="test")
label_list = get_unique_labels()
print("Number of sentences in training data: {}".format(train_df.shape[0]))
print("Number of sentences in testing data: {}".format(test_df.shape[0]))
print("Unique labels: {}".format(label_list))

Maximum sequence length in train data is: 746
Maximum sequence length in test data is: 2427
Number of sentences in training data: 45000
Number of sentences in testing data: 3442
Unique labels: ['O', 'B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER']


In [4]:
train_df.head()

,sentence,labels
0,当希望工程救助的百万儿童成长起来，科教兴国蔚然成风时，今天有收藏价值的书你没买，明日就叫你悔...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,藏书本来就是所有传统收藏门类中的第一大户，只是我们结束温饱的时间太短而已。,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,因有关日寇在京掠夺文物详情，藏界较为重视，也是我们收藏北京史料中的要件之一。,"[O, O, O, B-LOC, O, O, B-LOC, O, O, O, O, O, O..."
3,我们藏有一册１９４５年６月油印的《北京文物保存保管状态之调查报告》，调查范围涉及故宫、历博、...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,以家乡的历史文献、特定历史时期书刊、某一名家或名著的多种出版物为专题，注意精品、非卖品、纪念...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


### Tokenization and Preprocessing
The `tokenize_ner` method of the `Tokenizer` class converts raw string data to numerical features, involving the following steps:
1. WordPiece tokenization.
2. Convert tokens and labels to numerical values, i.e. token ids and label ids.
3. Sequence padding or truncation according to the `max_seq_length` configuration.

**Create a dictionary that maps labels to numerical values**

In [7]:
label_map = create_label_map(label_list)

**Tokenize input text**

In [8]:
tokenizer = Tokenizer(language=LANGUAGE, 
                      to_lower=DO_LOWER_CASE, 
                      cache_dir=CACHE_DIR)

**Create numerical features**  

In [9]:
train_token_ids, train_input_mask, train_trailing_token_mask, train_label_ids = \
    tokenizer.tokenize_ner(text=train_df[TEXT_COL],
                           label_map=label_map,
                           max_len=MAX_SEQ_LENGTH,
                           labels=train_df[LABEL_COL])
test_token_ids, test_input_mask, test_trailing_token_mask, test_label_ids = \
    tokenizer.tokenize_ner(text=test_df[TEXT_COL],
                           label_map=label_map,
                           max_len=MAX_SEQ_LENGTH,
                           labels=test_df[LABEL_COL])

１９４５
》，
”。
１９９７
（１９３７
—１９４５
”。
》（１９１９
）、
》（１９２３
）、
（１９２７
）、
》（１９２６
）、
》（１９３０
１９０８
）。
”，
”。
１９７４
”，
”。
”，“
”，“
”，“
……
”，“
”，
”，
，“
”。
”；
”。
１９６７
”、“
：“
”，
”，
”。
：“
：“
……”
……
”，
……
———
，“
：“
，“
，“
：“
：“
：“
：“
……
：“
”。
”。
、“
，“
”！
：“
”。
：“
———《
》，
……
”。
》，
》，
》，
……
”。
……
”。
，“
”。
———
》（
———《
———
》（
”，
：“
”，
”，
”；
”，
”。
”。
”，
”。
”，
”，
”，
———
，“
”，“
”，
”，
，《
———
……
１９３８
———
———“
”，
……
》，
：“
：“
１９９８
”。
，１９８３
１９７７
，１９７９
：<ADDREXTYPE="PHONE">（０１０）６４０１４４１１
２９０８</ADDREX>
”、“
”，
１０％—２０％
”（
８２）
”，
”，
：“
１９９７
”，
”，
”，
”。
：“
，“
”。
，“
”，
），
，“
，“
———
”。
２００
１９９７
１４００
１９９５
”。
１９９３
，１９７８
１９８５
１９８８
）（
』，
》。
……
：“
１９９１
”，
：“
：“
：“
……
……”
：“
：“
１９４０
———
———“
”，
”（
）。
”。
”，
”，
”。
，“
”，
”。
”，
”，
”，
———
———
：“
：“
……
……
１９９６
……
———
２６
”，
：“
；“
”。
，“１６”
２７，
１７
１０％
，９０％
１９９７
１．３３
２０００
０．０７
１９９７
３０３．９％。
１００
”。
１９９７
４０００
２０００—３０００
２００
０．７５
４．２
０．７５
１２．７
２００
９０
１９９７
———
———
；———
；———
２０％—３０％；———
４０％；———
１．５
，１９９１
３７
；———
３００
５００
———
１９５９
４０００
２００
２００
）、
）、
１９７８
：“
３００
１６０—２００
３５０
３３％。
１３００
２６
５００
１４６３
５．２
３．６
１／４，

１９９５
……
４∶０
２∶３
２∶２
１∶２
－－－
１９５
３４５
（１５０
），
７．５
７７
５８
８７．５
１２２．５
２１０
２３
１５∶１、１５∶８
１１∶８、１１∶８
１１∶４、１１∶０
１５∶１０、１５∶８
１１∶５、１１∶７
２１
５∶０
４∶１
１８６０
５∶４
：１９９８
，“
”。
，２６
，２２
３∶２
２∶３
、１１
２１
９０
）２１
、２２
１９
５∶２
；２０
４∶３
，５
１９
７∶１５、１３∶１５
１５∶１１、１５∶６
４∶１
”。
———
２１
１∶０
，４
１７
１９
１∶１
３∶２
４∶１
２２
：Ａ
；Ａ
２∶３
４∶１
３∶２
１∶２
１∶２
２∶１
”。
２∶０
１∶１
１６∶１７
０∶２
４∶８
１４∶１４。
１５∶３
，Ａ
－－－
６９
１４５
３１５
９０．５
９２．５
１１２．５
２０５
５３
１９８６
２１
２５
１７
”。
８１
》，５
”———
２∶３
１１∶６、１１∶３
２∶３。
”。
２∶３
２∶２
２∶０
３∶２
２１
５∶０
５∶０
１７
１∶２
———
１９
４∶１
５∶０
２１
：Ａ
；Ｂ
２∶０
”。
５∶０
３∶２
１９
：２１
１２∶３５
７∶００
１１∶４２
２１
１∶０
ＡＣ
１９
８３００
１９
（１８０
１４０
１７５
３１５
６２
５６
１２０
１５０
２７０
９５
２．５
４８
，１６
７０
２．５
”，
１９
０∶１
１∶０
１∶１
１７
１∶０
７７９０
８３００
）、
），
８３００
１∶３
４∶１、８∶０、２∶１
０∶０，
ＢＴＶ
５∶４
２６
３１
６３∶６９。
８７∶１１４
……（
６∶１５
１５∶９、１８∶１５
０∶３
１∶４
２∶０
０∶３
４∶１
５∶０
４∶１
１５∶５、１６∶１７、８∶１５
２∶０
５∶０
５∶０
２∶３
”，
”。
……
１１∶０、１１∶１
———
２００
１９９６
２３
—１９９７
２９
２０００
１∶１１
１１∶４
１∶１１、３∶１１
———
５∶０
３∶２
４∶１
４∶１
２∶０
２∶０
１１∶４
１∶１１、３∶１１
１∶２
１１∶５、１１∶７
４∶１
１７
，６２
２００２
———
３１
２∶２
２∶１
０∶０
０∶０
０∶２
２∶１
０∶１
１７
１∶０
２３
２１∶１３、２１∶１１
１２∶８，
２１∶１３
３∶２、２∶０

”。
４４
１９９６
１９９６
１９８４
８０
６２０
３７０
，５
”，
３９·５０
４０·５５
１７００·０３
７１·２９
１７２·４０
———
１６２·３７
１９９７
７５
３％。
４５０
６００
５６
２５０
８０
ＤＥＣ
●ＤＥＣ
ＤＩＧＩＴＡＬＰＣ
Ｐ６３００
ＧＢ１３０００／ＧＢＫ
１９９５
———
：“
８００
……
……
４０
２ＢＱ—１４０
７ＣＢ—２
１９９６
○○○
”。
：“
Ｘ—３３
Ｘ—３３
１６６６
８０
１９９２
Ｘ—３３
１９９８
Ｘ—３３
Ｘ—３３
：“
Ｘ—３３
Ｘ—３３
７５００
”。
１９７５
１９６５
”，
”，
”。
”。
———
５００
９００
”，
———
，“
”。
７００
８６
１００
９０
———
：“
８０
……
：“
１０００
”，
”、“
”，
》。
———
１９８６
１９９７
９０
”，
１／５。
８０
１３２
２００
４０
１９９７
７００
９０％
３１％，
５０—６０
１２０
、１０
、３９
２００２
”。
９０
ＢＢＬ
，５
，“
４２０
（１
１／３
３０００
８００
２２６４
１９９６
１５０
２％，
４０
２０％
１０％，
１．５％。
３．７５％
３％—３．２５％。
３００
３．５
１２００
２２００
５．２５
３４
１３００
；２４００
２９
１．７５
２．８
２２
１０３
２７
２１
）２、
４０
７０
１５００
２１
１、５
，１９９６
２·９６％。
《１９９７
１９９８
”。
１９９８
，１９９８
”。
１００
３４８．６１
３５１．８６
１９８９
４００
，４
３５０
１９８９
１·７７６６
１·７７６２。
１３２·７５
１３４·０７。
３８
４２０
———
４００
２／３
１９９９
２·８
“１９９８
”，
”，
３８
４０
，１９９９
２０００
，１５
２３
４６
４０％
５００
１．８
（６
（２０
，２９
，１９９８
，＊
８２５００
４．５
４４００
６％—２５％，
８０
１００
３．０３％
７３６
８０４
８５３
１９９７
４８
３ＡＷ
———“
”。
２、
１、
７·５
”。
１９８６
２６
３７０
１６５
５７００
１／３，
４０８·５
１９９６
———
（ＳＢＣ）１１
６２０
（Ａｍｅｒｉｔｅｃｈ），
”。
２１
４０％
２００２
，“
”，“
”。
２·９
１

３００
２８
２７
２８
８０
１９９８
８％
，５
２１
、２２
８％。
２８
２８
１９９８
２７
２８
》，
１９９７
１９９３
”，１９９６
１９９７
１９９６
”。
”。
１９９３
１９９８
１９９１
”，
３６
４３
３５
４２
２８
—４
４—６
２８
：２２
—２４
７℃—１０℃。
２８
：『
４—５
：２８
２９
（５
２８
—５
２９
２５
２１
２７
：『
３３
ＢＮＯ
１５０
２７
１９９７
１９９７
２７２
，１５
，３２
，５
》，
１９９７
”（
）。
２７
》１９９８
１９９７
，５
２６
２９
２７
２６
２７
”，
）、
”，
”。
２、
１９９６
１、
“６·２６”
２７
“６·２６”
１４℃／２７℃
１８℃／２６℃
１８℃／２６℃
９℃／２９℃
９℃／２２℃
１２℃／２１℃
１６℃／２０℃
８℃／２２℃
８℃／１８℃
１９℃／２９℃
２０℃／３０℃
１８℃／３０℃
１７℃／２５℃
２２℃／３０℃
１８℃／２８℃
１５℃／２７℃
１３℃／２２℃
１６℃／２９℃
２０℃／２６℃
２０℃／３０℃
２３℃／３１℃
２２℃／３３℃
２６℃／３３℃
２０℃／２７℃
２２℃／３０℃
１５℃／２２℃
１６℃／２５℃
６℃／２６℃
１４℃／２６℃
１２℃／２８℃
９℃／２５℃
１３℃／２７℃
１７℃／２３℃
２２℃／３０℃
２２℃／３０℃
２２℃／３０℃
１４℃／２５℃
２６℃／３４℃
１１℃／２０℃
２７℃／３５℃
１９℃／３３℃
９℃／２０℃
１０℃／１８℃
１１℃／１７℃
８℃／１５℃
１３℃／２４℃
２７
２８
５—６
（５
２７
—５
２８
”，
”，
２６
”，
”、“
”、“
”，
１９８８
１９９２
７％，
２６％。
９５７
》，
２６
”、
２６
”（
２６
２６
２６
』。
：『
』。
……
』。
，『
，『
』，
，『
１９９６
４０
１５℃／２７℃
１６℃／２６℃
１８℃／２７℃
１２℃／２９℃
７℃／１９℃
１５℃／２５℃
１４℃／１９℃
８℃／２０℃
１０℃／２５℃
２０℃／３０℃
１７℃／２７℃
１７℃／２９℃
１８℃／２９℃
２１℃／２５℃
２０℃／２９℃
１８℃／２７℃
１５℃／２１℃
１７℃／２８℃
２０℃／３０℃
２０℃／３０℃
２４℃／３１℃
２３℃／３３℃
２４℃／３０℃
２０℃／２６℃
２０℃／３１℃

』。
４—６
５—７
４—５
（５
—５
６０
，“
５１０
，１９９９
１９９８
（１９９８
２１
１９９６
２１
２１
６．
５．
；４．
；３．
；２．
；１．
２—３
５．
；４．
；３．
；２．
４．
３．
；１．
”）
１２℃／２４℃
１３℃／２１℃
１５℃／２４℃
８℃／２１℃
８℃／１９℃
１３℃／２２℃
１１℃／１８℃
９℃／１７℃
１０℃／２１℃
１８℃／２５℃
１８℃／２６℃
１９℃／２６℃
１９℃／２９℃
２２℃／２９℃
２２℃／３０℃
１６℃／２４℃
１１℃／１７℃
１７℃／２３℃
２０℃／２８℃
２０℃／２８℃
２５℃／３２℃
２４℃／３３℃
２５℃／３２℃
１９℃／２６℃
２２℃／２８℃
１７℃／２３℃
１６℃／２７℃
７℃／２３℃
１５℃／２４℃
１３℃／２５℃
９℃／２２℃
６℃／２４℃
７℃／２０℃
２２℃／２７℃
２５℃／３１℃
２４℃／３１℃
１５℃／２４℃
２７℃／３６℃
１４℃／１８℃
２６℃／３５℃
２３℃／３７℃
６℃／１６℃
８℃／１５℃
７℃／１４℃
８℃／１６℃
１５℃／２１℃
，６
（５
—５
，５
６０％
》，
１９８８
，１９９３
３１
１９８７
１９９９
，１６
１００
１９５４
３００
：『
２５
，“
１９９９
１９９９
”、
”，
２００
１３℃／２０℃
１５℃／２４℃
１５℃／２５℃
１０℃／２５℃
８℃／１４℃
１４℃／１８℃
１１℃／１７℃
１２℃／１８℃
１２℃／１８℃
１７℃／２７℃
１７℃／２７℃
１７℃／２４℃
１９℃／２９℃
２１℃／２７℃
２１℃／２９℃
１８℃／２８℃
１１℃／１８℃
１６℃／２７℃
２０℃／３１℃
１９℃／３２℃
２５℃／３０℃
２４℃／３４℃
２５℃／３３℃
２２℃／２８℃
２１℃／３１℃
１７℃／２６℃
１７℃／２８℃
５℃／２０℃
１４℃／２２℃
１３℃／２２℃
９℃／２０℃
７℃／２１℃
３℃／１４℃
２３℃／２８℃
２４℃／２９℃
２４℃／２９℃
１３℃／２４℃
３０℃／３６℃
１３℃／２２℃
２５℃／３４℃
２０℃／３６℃
１３℃／２１℃
１１℃／１７℃
７℃／１５℃
６℃／１５℃
１１℃／１７℃
５—６
（５
—５
３７
２５
”，
”、“
”。
１９９６
２３
２８
９００
７０
１００
１９８６
８１
８４％，
２３
》、《
３０１
３０１
”

...

１７
２１
２１
”，
１７
，１４
）２１
２１
２１
２２
）。
”。
２６
２５
，“
”，
”、“
”，
：“
２１
”，
”。
”。
５００
２０００
２０００
１９５６
１９９３
，《
１９５６
１９９３
１９９３
”。
０．３％，
”，
２∶０
１９
６６
７３％。
１９
６３·５％，
１４％。
１９
１５０
１９９８
１９
１９
５５
６０００
１９８０
１９８８
４００
１９
１７
１７
２２
，２６
１７
１９７２
１９６０
１７
１７
ＣＮＮ、《
》、《
２００
……
ＷＴＯ、
』－－－
”。
１７
１７
１７
，６
１７
１７
１７
１７
１７
１７
１７
１７
”，“
”。
１７
１９９１
１９９３
１９９６
，“
８０
２００
４２
；《
……
》，
———
，７４％
１９４８
１９６７
１７
，５４０
９７％
３９
１２２
：１·
；２·
；３·
《２０００
》。
》，
）６
，１５
”、“
２０００
１９８４
１７
，“
，“
”，
３０％
”，
”。
”。
，１５
４％
６％
”；
，１５
”。
１７
”，
》１７
１７
》、《
》、《
１９７２
１７
、５
、５
”，
，１５
，“
８３
２０００
２０００
，１９９７
３０％，
２％。
１５００
，２０００
３／４
２００
—３００
２０００
３７
１９９３
２０００
－－－
”。
１９
”。
”，
２７
”。
”。
，１００
１００
２２
３８．７
３５０
１００
”，
”。
６０
２００
６８７
２２
”。
”。
”、“
１９９５
１９８９
８０
”。
：“
，７０—８０
９０
１．２
８０００
”，
Ｆ—１６
，“
”。
，“
”。
，１９９１
”。
，“
”。
，１３
８０
”，
３０００
：“
”，
２８
１７５
３５
）３５
３０００
７５
）。
２０７
、２７３
，“
———
”。
”。
”。
，“
”。
”。
：１３
：“
、《
》、《
……
”。
１２００
———
”。
，２０
１９９８
９３
５００
１９９０
１００
１００
２１
１００
２１
”。
３６
１９８８
１９９６
１９８８
２：
６７
、３０
１：
１．１８
，８０％
），
———
———
１．２
１３０
１００
１９９５
１９９７
２．６
１．３
”，
３１
４６００
１９９７
》、《
１９
１

２０％
２１
２１
－－－
１９９６
３９０
１０００
３４３
５３０４２
，１９９６
２０％，
７５％。
１９９６
８０
，１９８３
５２％，７５％
６５
１９９６
１００
１９９７
”（
）、“
”（
）、“
”（
：１９９６
》、《
》、《
》、《
》、《
》，
》。
２∶３
３∶０
１５∶１２
１５∶８
１∶５
１５∶１１
１７∶１６
８∶４、１３∶９
、１４
、１５
、１６
１∶３
３∶２
：“
，“
”。
”。
，“
，“
：“
”“
：“
……”
：“
３５０
２００
：“
：“３００
“ＦＩＬＡ”。
：“
，３５０
１２００
”。
：“
１４５
“ＦＩＬＡ”
：“
”６
”。
”。
”。
，“
”，
”。
”，
”，
１∶０
２８
”。
３００
”，
４０
７５
》６
１９
》，
》，
》，
》。
９７／９８
０∶１
０∶０
１∶０
２８
”。
３００
１００
９７％
１９９５
１９９５
４０００
１９９１
：“
’，‘
７０
１９８８
５００
２０００
６２
———
５００
２１
———
４３
２６
，２０
２００
》、《
》、《
３４
４３
４３
２４００
１９９５
４３
———
———
———
———
》。
”、“
”、“
》，
）。
』。
”，
”。
”，
”。
”，
”，“
”、“
”，“
”，
、“
，《
，《
：『
：『
』。
』。
：『
》。
，《
》，
》、《
》、《
》、《
》、《
》、《
》、《
－－－
”，
”。
”，
”，
”。
”，
”。
１９８６—１９９７
，“
，１９１９
８０
”、“
１８４０
１９４９
４０００
”。
”。
１９９５
”。
，１
———
１００
”、“
”、“
”、“
《２１
１００
：１１８
１００
９４．５％，
４８．５
２２．６
６１
１９９５
，１１
———
９８％
５８００
”，
１４０００
３０４
１９９７
４５
７．３６
１９．５０％。
”（
２％
３％；
，１９９７
６８．６５
１９．６８％。
２２
１０００
５３０
２４００
３４００
６０
２５％，
５１％，１９９７
１９９２
５０００
４０００
４０００
６０００
》、《
》、《
》，
》、《
》、《
》。
”，
１９９１
，《
———
２４０
６１．９％，
”。
４２
９０％
》。
１９９７
……
；１９９５
、１９９６
２１

１７
２８
———
』『
』『
』，
』。
』，
』，『
』，
』？
２２℃／２８℃
２２℃／３３℃
２１℃／３４℃
１８℃／３２℃
１６℃／２８℃
１６℃／２６℃
１８℃／２４℃
１６℃／２６℃
１４℃／２７℃
２１℃／２９℃
２０℃／２９℃
２１℃／２７℃
２０℃／２７℃
２６℃／３３℃
２１℃／２４℃
２４℃／３４℃
１７℃／２５℃
２５℃／３０℃
２３℃／２９℃
２３℃／２８℃
２７℃／３４℃
２９℃／３７℃
２８℃／３７℃
２２℃／３０℃
２３℃／２９℃
１８℃／２６℃
１８℃／２８℃
１５℃／２８℃
２２℃／２８℃
２０℃／３２℃
１５℃／２３℃
１８℃／２７℃
１７℃／２８℃
２７℃／３４℃
２８℃／３４℃
２８℃／３４℃
１６℃／２１℃
２６℃／３４℃
１１℃／２０℃
２７℃／３６℃
２０℃／３９℃
１６℃／２６℃
１１℃／２０℃
１２℃／２２℃
１２℃／２１℃
１４℃／２０℃
１７
（６
—６
１７
，５
１％。
３０％、
３０％
１—５
２６．５
２７．１１％，
１４．５
１３．６２
：５
”８
：“
３００
２００
１００
２００
”“
２７
２００
１７
４５
３８
，３
１１５２５
，３３１７８
４００
”，
”、“
———
》。
”，
１９９４
ＤＴＤ
３００
，５０
１９８９
１９７７
———
”（
）（
１·５
１·２
３０００
３５
２、
７００
１、
２１
３０４５．６９
７．９％，
３７．６％，
４．１
１５１７．８３
１％，
３４．１％，
９．１
７．６
８００
１０００
８、
７、
６、
５、
４、
３、
２、
３２
１、１９９６
２６
、２０
１９９６
７０００
：“
２１
３２
１５０
２０％
”、“
”、“
”、“
“２１
，《
２００
３８—３９
１００
》（
１１０
１９９７
：６
，１１
”。
４．２
１１００
１０００
”，
４．６
：６
，１９９６
７．５
，５２．７
２００
２００
５２．７
１９９２
２６３
２６８
》，
”，
》，
”，
”，
》、《
（１９９６）》、《１９５８
１９９３
９９．８％，
６６％
５６．２％。
８５％
，１９９７
９９％
３９％，
５４％，
６０％。
７３％
１９７８—１９９６
７０％
８０％
９０％
ＧＤＰ
，１９９６
９９．７％，１９９１—１９９６
７６．６％。
：“


２０１１
８１０
２２
３６
２００１
，６００
０．０３％，
（“
１９９９
１９９８
”、“
”、
４００
，１９９７
１０９
，６３１
１９９７
４３００
３５
３１
２８２１２
１９９７
８５０
１９９６
４９．１２％，
２．９４
１９９６
７９．５３％，
２．７５
１９９６
９．６９％。
２００
１９９７
６３６６
１００．６４
３７．４
，１９９７
２６６
１９９６
６．６％，
３．６９
１９９６
２２．３％，
４．０３
３３．５％。
３１％，
６％。
，１９９７
７．６４
１９９６
１０．２％，
６．４
１９９６
１２％；
１．２
１９９６
１．６％。
６０８
９８
７８１７４
１８７６
２．４％。
５３
２３２
１２００
６０％
４４
６０％
１．８
》，
１７
６６％
２７２０
４８５０
”，
，１
，２２
５１８
”。
”。
１１１
』，
』，
４１００
３０００
１００
，１１００
：“
３０００
２００
、１００
１００
１９９５
５００
１９９４
：“
１９
……
９８６
、２００
２８
：“
———“
”。
１９９５
３９
》６３５
》。
”，
７４
３１
２５
１．２５
１８００
１９９４
２００
１９９６
４０００
１５５０
７０１
３１
８０
，５０００
４００
３１
１０８
”。
５００
３０％
，３００
３０００
４０００
１９９５
－－－
”，
：“
”、“
”、“
”；
：“
”；
”……
……
”，
，“
，“
，“
○”
３４
１１００
１０００
１３６
１９９３
１１００
○”
１９９５
５００
”，
”，
”，
６０
６０００
１９９５
１５０
，１９９２
○”
○”
５４
４７
，“
”。
１０００
：４
１９８９
：“
”２０
，“
”、“
”……
１９９８
）“
３４８０Ｗ（３０００ｋｃａｌ／ｈ）
７．５Ａ，
１Ａ
１Ａ
）。
２０％—４０％
×１４０Ｗ。
２．８
，Ｆ
，Ｃ
，Ｒ
，Ｄ
×１００Ｗ。
５２．９％
，３６．７％
６０．１％，
１０．４％。
３６．５％；
２２．１％；
２０．３％
１５．６％；
５．５％。
２２％
”，
９．８％
５８．７％
６６％
４．
３．
２．
１．
１１４
３０％，
３％。
７５０
８８０
；２０００
１１５０
１５００
４．
２．５—７
１０—３０
Ｗ。
１０％。
２０％

——
》（
———
：“
、“
”、“
”，
”，
』。
』，
』，
……
”，
》（
）、《
》（
）、《
》（
）、《
》（
）、《
》（
》（
）、《
》（
）、《“
》（
）、《
》（
》（
）、《
》（
）、《
》（
）、《
》（
）、《
》（
）、《
》（
———
》，
》、《
》、《
》，
』，
』（
》、《
』，
（《
》，
１∶０
１∶１，
”，
”、“
”、“
：“
……”
：“
：“
：“
”，７
”，８
”，１０
”……
”。
》，
》，
》，
》，
”。
》，
：“
：“
”（
５５
《“
，《
：“
：“
：“
：“
，“
１９９０
：———
———
４０９５
２６．７４％；
５２０
２４１５
６８３
———
１３７４０１２
５７８
２５２５３８
”、“
”、
１５０
１９８４
１９９０
１９９７
，１９９０
１９９１
１９９７
１９９５
１９９５
》；
》，
１９９７
４９
１９４８
，１９７０
，１９７３
，１９８８
１９９１
１９９２
１００
１１０
”（
”。
”、
ＳＯＳ
ＳＯＳ
１００
２７
１００
１００
”。
２１
，５２３
３００
１０２
８８
１０２
８８
：———
６４
１１０
１１０
》，
；———
，５８
５０９８
；———
、８８
；———
５７０
３７９
５６０９
；———
２５７３
６５１２
４０％
５４００
３１００
６２
５６０
——
１９８２
１９９０
”，
１００
”。
３６５
”，
”，
：“
”，
”，
３０％，
４８％，
１．６
４８５９
４０６５
３３１１
２６８１
１９５９
１１２８．４
７０７
）３４２
５１９．４
４７０
３３
１３．６
”、“
”、“
”（
）、
”。
○、
○，
』，
○、
——
６１
３１５
３０４７
１７５２
１３６４
８０８
７０％，
６０％
２３
１９９６
１０２
１００
１９７８
２．６３
１０２
３８．８
１００
”。
，“
２４８７
４％
２０１０
》，
９７
：“
”。
２１
ＰＡＴＡ）
５１００
１０２
６．４
１６３８
９７
ＰＡＴＡ
、ＰＡＴＡ
１５００
２００
１９５２
２０００
１．８
１９９３
ＰＡＴＡ，
１９９４
”，
，ＰＡＴＡ
ＰＡＴＡ
——
”、“
１００
”……
”、
”，
２５
……
，“
１９
４０
……”
６０
７０
１９

`Tokenizer.tokenize_ner` outputs three or four lists of numerical features lists, each sublist contains features of an input sentence: 
1. token ids: list of numerical values each corresponds to a token.
2. attention mask: list of 1s and 0s, 1 for input tokens and 0 for padded tokens, so that padded tokens are not attended to. 
3. trailing word piece mask: boolean list, `True` for the first word piece of each original word, `False` for the trailing word pieces, e.g. ##ize. This mask is useful for removing predictions on trailing word pieces, so that each original word in the input text has a unique predicted label. 
4. label ids: list of numerical values each corresponds to an entity label, if `labels` is provided.

In [8]:
print(("Sample token ids:\n{}\n".format(train_token_ids[0])))
print("Sample attention mask:\n{}\n".format(train_input_mask[0]))
print("Sample trailing token mask:\n{}\n".format(train_trailing_token_mask[0]))
print("Sample label ids:\n{}\n".format(train_label_ids[0]))

Sample token ids:
[2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

## Create Token Classifier
The value of the `language` argument determines which BERT model is used:
* Language.ENGLISH: "bert-base-uncased"
* Language.ENGLISHCASED: "bert-base-cased"
* Language.ENGLISHLARGE: "bert-large-uncased"
* Language.ENGLISHLARGECASED: "bert-large-cased"
* Language.CHINESE: "bert-base-chinese"
* Language.MULTILINGUAL: "bert-base-multilingual-cased"

Here we use the base model pre-trained only on Chinese data.

In [9]:
token_classifier = BERTTokenClassifier(language=LANGUAGE,
                                       num_labels=len(label_map),
                                       cache_dir=CACHE_DIR)

## Train Model

In [ ]:
token_classifier.fit(token_ids=train_token_ids, 
                     input_mask=train_input_mask, 
                     labels=train_label_ids,
                     num_epochs=NUM_TRAIN_EPOCHS, 
                     batch_size=BATCH_SIZE, 
                     learning_rate=LEARNING_RATE)

t_total value of -1 results in schedule not being applied
Iteration:   0%|          | 0/2813 [00:00<?, ?it/s]


Iteration:  18%|█▊        | 504/2813 [10:51<49:59,  1.30s/it]

## Predict on Test Data
The `predict` method of the token classifier optionally returns the softmax probability of the predicted class, which is a NxM array, where N is size of the testing data and M is the number of tokens in the testing sequence

In [ ]:
predictions = token_classifier.predict(token_ids=test_token_ids, 
                                       input_mask=test_input_mask, 
                                       labels=test_label_ids, 
                                       batch_size=BATCH_SIZE,
                                       probabilities=True)
probabilities = predictions.probabilities
pred_label_ids = predictions.classes
print("Sample predicted probabilities:\n{}\n".format(probabilities[5]))
print("Sample predicted labels:\n{}\n".format(pred_label_ids[5]))

## Evaluate Model
The `predict` method of the token classifier outputs label ids for all tokens, including the padded tokens. `postprocess_token_labels` is a helper function that removes the predictions on padded tokens. If a `label_map` is provided, it maps the numerical label ids back to original token labels which are usually string type. 

In [ ]:
pred_tags_no_padding = postprocess_token_labels(pred_label_ids, 
                                                test_input_mask, 
                                                label_map)
true_tags_no_padding = postprocess_token_labels(test_label_ids, 
                                                test_input_mask, 
                                                label_map)
print(classification_report(true_tags_no_padding, pred_tags_no_padding, digits=2))

`postprocess_token_labels` also provides an option to remove the predictions on trailing word pieces, e.g. ##ize, so that the final predicted labels correspond to the original words in the input text. The `trailing_token_mask` is obtained from `tokenizer.tokenize_ner`

In [ ]:
pred_tags_no_padding_no_trailing = postprocess_token_labels(pred_label_ids, 
                                                            test_input_mask, 
                                                            label_map, 
                                                            remove_trailing_word_pieces=True, 
                                                            trailing_token_mask=test_trailing_token_mask)
true_tags_no_padding_no_trailing = postprocess_token_labels(test_label_ids, 
                                                            test_input_mask, 
                                                            label_map, 
                                                            remove_trailing_word_pieces=True, 
                                                            trailing_token_mask=test_trailing_token_mask)
print(classification_report(true_tags_no_padding_no_trailing, pred_tags_no_padding_no_trailing, digits=2))

You can see that the model performance is the same before and after removing trailing word pieces. This is because Chinese characters/words don't contain word pieces as English does.